ADP data by Steven Cunningham 06/16/17

This code is intened for take SonTek ADP data
- First determine seafloor by signal amplitude file ".a#". The signal should slowly decay through water, when decay is
  interupted with amplitude spike, then the signal is bouncing back from the floor. 
- Compare one bin to the previous until the next bin is larger than previous.
- Number of good profiles will be used on velocity data. 

In [1]:
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt
from scipy import stats
import scipy
import csv


In [3]:
sig_amp = 'SWC0610130945.a1'
df = pd.read_csv(sig_amp, delimiter='\s+', header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,1,147,132,117,110,104,105,121,158,169,...,33,30,29,31,37,42,42,34,29,27
1,2,149,133,118,110,107,109,118,159,176,...,32,28,27,28,34,43,44,32,26,24
2,3,148,137,124,112,107,114,125,156,175,...,33,29,27,28,33,40,45,33,26,24
3,4,160,152,123,109,101,99,104,139,183,...,36,30,27,26,30,39,48,35,26,24
4,5,152,147,132,113,106,104,104,149,182,...,36,29,26,26,30,44,48,33,26,24


In [66]:
mean = np.mean(df, axis = 0)[1:] # average profiles across a depth bin
mean2 = pd.Series.to_frame(mean, 'average_profile') # transform to dataframe 
mean2["next_profile"] = mean2["average_profile"].shift(-1) # Shift down to commpare difference 
mean2["Nartural_decay"] = mean2["average_profile"] > mean2["next_profile"] # check if the next bin is greater than previous 
mean2["True_profile"] = mean2["Nartural_decay"].cumsum() # gives values to groups of true/false
mean2["Number"] = np.arange(23)[1:] # make a range of values 
mean2["good_profile"] = mean2["True_profile"] == mean2["Number"] # see if values assigned to true/false match number order  
#print(mean2)
how_many=sum(mean2["good_profile"]) # how many good profiles are there. This will be used on velocity data 
mean3=mean2["average_profile"][:how_many+1] # get profile values for good bins. Add 1 because the .shift 


1    150.357143
2    136.642857
3    122.071429
4    113.500000
5    108.214286
Name: average_profile, dtype: float64
